In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [52]:
sc = spark.sparkContext
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [70]:
df_data = sc.textFile("/labs/laba01/ml-100k/u.data")

In [71]:
# df_item = spark.read.text("/labs/laba01/ml-100k/u.item")

In [72]:
df_data_2 = df_data.map(lambda x: x.split("\t"))\
          .map(lambda x: [int(a) for a in x])\
          .map(lambda x: ((x[1], x[2]), x[2]))\
          .groupByKey()\
          .mapValues(len)

df_data_3 = df_data_2.filter(lambda x: x[0][0] == 96)\
           .map(lambda x: (x[0][1], x[1]))\
           .sortByKey(True)\
           .map(lambda x: x[1])\
           .collect()

df_data_4 = df_data_2.map(lambda x: (x[0][1], x[1]))\
           .groupByKey()\
           .mapValues(sum)\
           .sortByKey(True)\
           .map(lambda x: x[1])\
           .collect()

In [73]:
data = {}
data["hist_film"] = df_data_3
data["hist_all"] = df_data_4
data

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [74]:
with open('./lab01.json', 'w') as outfile:
    json.dump(data, outfile)

In [75]:
sc.stop()